In [1]:
from pathlib import Path
from collections import deque
from itertools import repeat, permutations
from io import StringIO
import numpy as np
import re
from functools import cache

In [2]:
codes = Path("data/21.txt").read_text().strip().split("\n")


def ascii_to_grid(ascii_data: str):
    grid = []
    for line in ascii_data.split("\n"):
        if "|" in line:
            values = [val.strip() for val in line.split("|")[1:-1]]
            if len(values) == 2:
                values = ["#"] + values
            grid.append(values)
    return np.array(grid, dtype=object)


def neighbors(x, y, grid, dx0, dy0):
    dx0, dy0 = 1, 0
    for dx, dy in [(dx0, dy0), (-dx0, -dy0), (dy0, dx0), (-dy0, -dx0)]:
        if (
            0 <= x + dx < grid.shape[0]
            and 0 <= y + dy < grid.shape[1]
            and grid[x + dx, y + dy] != "#"
        ):
            yield x + dx, y + dy


def l1(x0, y0, x1, y1):
    return np.abs(x0 - x1) + np.abs(y0 - y1)

In [3]:
numeric_keypad = ascii_to_grid(
    """
+---+---+---+
| 7 | 8 | 9 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
    | 0 | A |
    +---+---+
""".strip()
)

numeric_lookup = {v: (i, j) for (i, j), v in np.ndenumerate(numeric_keypad) if v != "#"}

directional_keypad = ascii_to_grid(
    """
    +---+---+
    | ^ | A |
+---+---+---+
| < | v | > |
+---+---+---+
""".strip()
)

directional_lookup = {
    v: (i, j) for (i, j), v in np.ndenumerate(directional_keypad) if v != "#"
}

In [4]:
dirs = {(0, -1): "<", (0, 1): ">", (-1, 0): "^", (1, 0): "v"}


def pretify(p):
    return [dirs[(t[0] - f[0], t[1] - f[1])] for f, t in zip(p, p[1:])]


def shortest_paths(a, b, grid):
    q = deque([[a]])

    while q:
        p = q.popleft()
        d = l1(*p[-1], *b)
        if len(p) >= 2:
            dx, dy = p[-1][0] - p[-2][0], p[-1][1] - p[-2][1]
        else:
            dx, dy = None, None

        for n in neighbors(*p[-1], grid, dx0=dx, dy0=dy):
            new_d = l1(*n, *b)
            if new_d == 0:
                yield pretify(p + [b])
            elif new_d > d:
                continue
            q.append(p + [n])

In [5]:
def keys(code, grid, lookup, acc=None):
    if len(code) <= 1:
        yield acc
        return

    start, goal = code[0], code[1]
    if start != goal:
        for p in shortest_paths(lookup[start], lookup[goal], grid):
            yield from keys(code[1:], grid, lookup, (acc or []) + p + ["A"])
    else:
        yield from keys(code[1:], grid, lookup, (acc or []) + ["A"])

In [6]:
def extract_int_from_string(text):
    match = re.search(r"\d+", text)
    return int(match.group()) if match else None


@cache
def min_len(code):
    m = None
    for p0 in keys(code, numeric_keypad, numeric_lookup):
        for p1 in keys(["A"] + p0, directional_keypad, directional_lookup):
            for p2 in keys(["A"] + p1, directional_keypad, directional_lookup):
                m = min(m or len(p2), len(p2))
    return m


sum(
    min_len(f"{a}{b}") * extract_int_from_string(code)
    for code in codes
    for a, b in zip(f"A{code}", code)
)

203734